<a href="https://colab.research.google.com/github/Master-Chief131/ShopSmart/blob/main/shopsmart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Realizar Scraping**
## Ahora vamos a realizar un script para extraer todos los productos de la pagina de super 99, y lo haremos navegando por las categorias

Como primer paso vamos a ver como clasifica sus productos super 99, para eso vamos a extrar el menu de su pagina principal

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import csv

Primero definimos la url base y parseamos el archivo html para poder navegar por su contenido

In [2]:
# Pagina de super 99
URL_BASE = "https://www.super99.com"
respuesta_obtenida = requests.get(URL_BASE)
html_home = respuesta_obtenida.text

# Parsear el html
super99_home = BeautifulSoup(html_home, "html.parser")

Ahora debemos buscar el contenido del menu y buscar su enlace para realizar una busqueda de todos los productos por pagina

In [3]:
categorias = super99_home.find_all("a", class_="level-top")

categoria = []

for enlace in categorias:
    categoria.append(enlace["href"])

Una vez tenemos los enlaces a las paginas debemos ir a cada categoria y obtener sus productos

In [4]:
# usar el primer enlace
Url_productos = categoria[0]
respuesta_productos = requests.get(Url_productos)
html_productos = respuesta_productos.text

# Parsear el html
super99_productos = BeautifulSoup(html_productos, "html.parser")

Revisamos cuantas paginas tiene esa categoria

In [5]:
# El enlace a la ultima pagina de esa categoria
paginas = super99_productos.find("a", class_="page last")

# devuelve dos span
spans = paginas.find_all("span")

span_sin_clase = [span for span in spans if not span.get("class")]

# utilizamos el bucle for para extraer el span con el numero y lo convertimos a entero
if span_sin_clase:
    numero_pagina = int(span_sin_clase[0].text.strip())
    print(f"Número de la última página: {numero_pagina}") # Lo usaremos para iterar las paginas
else:
    print("No se encontró un <span> sin clase.")

Número de la última página: 6


Primero descarguemos las imagenes

In [6]:
# Buscar las imagenes
src_productos = super99_productos.find_all("img", class_="product-image-photo")

nombre_productos = []
url_imagenes = []

for i, imagen in enumerate(src_productos):
    url_imagenes.append(imagen["src"])
    nombre_productos.append(imagen["alt"].strip())

In [7]:
# Las url tienen atributos incrustados
# Limpiar las url
cleaned_urls = [re.sub(r'\?.*', '', url) for url in url_imagenes]
cleaned_names = [re.sub(r"/", "-", re.sub(r" ", "_", name)) for name in nombre_productos]

Para poder distinguir las imagenes las guardaremos con el nombre del producto

In [8]:
# Descargamos las imagenes y las enumeramos
for i, imagen in enumerate(cleaned_urls):
  nombre_p = cleaned_names[i]
  r = requests.get(imagen)
  with open(f"{nombre_p}.png", "wb") as f:
    f.write(r.content)

In [ ]:
import os

# Crear la carpeta si no existe
if not os.path.exists('imagenes'):
    os.makedirs('imagenes')

for i, imagen in enumerate(cleaned_urls):
    nombre_p = cleaned_names[i]
    r = requests.get(imagen)
    with open(f"imagenes/{nombre_p}.png", "wb") as f:
        f.write(r.content)

Ahora buscamos el nombre y el precio de los productos

In [9]:
# extraer el nombre y el precio de los productos
producto_info = super99_productos.find_all("div", class_="details-info-product")

producto_nombre = []
producto_precio = []


for info in producto_info:
    nombre = info.find("a", class_="product-item-link").text.strip().replace("/", "-").replace(" ", "_")
    precio = info.find("span", class_="price-wrapper").text.strip().replace("$", "")
    producto_nombre.append(nombre)
    producto_precio.append(precio)

In [10]:
producto_nombre.insert(0, "Producto")
producto_precio.insert(0, "Precio")

In [11]:
datos = dict(zip(producto_nombre, producto_precio))

In [12]:
with open('datos.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    for key, value in datos.items():
        writer.writerow([key, value])

# Prueba de visualizacón de datos

In [13]:
import pandas as pd

# Leer el archivo CSV
df = pd.read_csv('datos.csv')

# Mostrar todos los datos
print(df)

# Si quieres ver los primeros 10 productos
#print(df.head(10))

                                             Producto  Precio
0            KONTOS_FLATBRD_ESTILO_GRIEGO_VEGANO_10OZ    2.99
1                KONTOS_FLATBREAD_VEGANO_CEBOLLA_14OZ    2.85
2             KONTOS_FLATBREAD_MUTLIGRANO_VEGANO_14OZ    2.85
3                 KONTOS_PITA_VEGANO_WHOLE_WHEAT_14OZ    2.85
4                 KONTOS_PITA_VEGANO_TRADICIONAL_14OZ    2.85
5                            Bimbo_Pan_De_Huevo_480_G    2.45
6                     Rimith_Pan_De_Huevo_Gande_624_G    3.95
7        La_Sabrosita_Pan_De_Hot_Dog_454_G_8_Unidades    1.49
8   Rimith_Pan_de_Hamburguesa_con_Ajonjolí_454_g_8...    1.99
9              Krispy_Kreme_Donas_Variadas_6_Unidades    9.54
10   Krispy_Kreme_Donas_Glaseadas_Original_3_Unidades    4.47
11  Bimbo_Pan_De_Hamburguesa_Con_Ajonjolí_350_G_4_...    1.79
12   Krispy_Kreme_Donas_Glaseadas_Original_6_Unidades    8.94
13                 PANAMA_BAKERY_HAMBURGUESA_8UN_450G    1.85
14                  La_Sabrosita_Pan_Hamburguesa_454G    1.49
15      

In [ ]:
# Ordenar por precio (si es numérico)
df['Precio'] = df['Precio'].str.replace('$', '').astype(float)
print(df.sort_values('Precio'))

Visualizar en una pagina html generada

In [14]:
import pandas as pd

# Leer el CSV
df = pd.read_csv('datos.csv')

# Convertir a HTML
html_table = df.to_html(index=False, classes='table table-striped')

# Crear página web completa
html_content = f'''
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <title>Productos Super99</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.2.3/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body>
    <div class="container mt-5">
        <h1>Productos Super99</h1>
        {html_table}
    </div>
</body>
</html>
'''

# Guardar página web
with open('productos_html.html', 'w', encoding='utf-8') as f:
    f.write(html_content)

Utilizando flask

In [15]:
from flask import Flask, render_template
import pandas as pd

app = Flask(__name__)

@app.route('/')
def mostrar_productos():
    # Leer CSV
    df = pd.read_csv('datos.csv')

    # Convertir a diccionario para pasar a template
    productos = df.to_dict('records')

    return render_template('productos_flask.html', productos=productos)

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


Con un plotly

In [17]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

# Leer CSV
df = pd.read_csv('datos.csv')

# Imprimir los tipos de datos para verificar
print(df.dtypes)

# Convertir la columna 'Precio' a string explícitamente
df['Precio'] = df['Precio'].astype(str)

# Limpiar datos de precio
df['Precio'] = df['Precio'].str.replace('$', '').astype(float)

# Crear gráfico de barras
fig = px.bar(df, x='Producto', y='Precio', title='Precios de Productos Super99')

# Convertir a HTML
grafico_html = pio.to_html(fig, full_html=False)

# Página web completa
html_content = f'''
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <title>Productos Super99</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.2.3/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body>
    <div class="container mt-5">
        <h1>Productos Super99</h1>
        {grafico_html}

        <table class="table table-striped mt-4">
            <thead>
                <tr>
                    <th>Producto</th>
                    <th>Precio</th>
                </tr>
            </thead>
            <tbody>
                {df.to_html(columns=['Producto', 'Precio'], index=False, classes='table')}
            </tbody>
        </table>
    </div>
</body>
</html>
'''

# Guardar página web
with open('productos_grafico.html', 'w', encoding='utf-8') as f:
    f.write(html_content)

Producto     object
Precio      float64
dtype: object


In [18]:
import csv

# Leer los datos desde el archivo CSV
productos = []
with open('datos.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        productos.append({"nombre": row[0], "precio": row[1]})

# Generar el HTML
html_content = '''<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Productos Super99</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f7f7f7;
        }
        .container {
            display: flex;
            flex-wrap: wrap;
            justify-content: center;
            margin: 20px auto;
            max-width: 1200px;
        }
        .card {
            background-color: white;
            border: 1px solid #ddd;
            border-radius: 5px;
            margin: 10px;
            padding: 20px;
            width: 200px;
            text-align: center;
        }
        .card img {
            max-width: 100%;
            height: auto;
            border-bottom: 1px solid #ddd;
            margin-bottom: 15px;
        }
        .card h2 {
            font-size: 1.2em;
            margin: 10px 0;
        }
        .card p {
            color: #333;
            font-size: 1em;
        }
    </style>
</head>
<body>
    <div class="container">
'''

# Añadir los productos al HTML
for producto in productos[1:]:
    nombre = producto["nombre"]
    precio = producto["precio"]
    imagen = f"{nombre}.png"
    html_content += f'''
        <div class="card">
            <img src="{imagen}" alt="{nombre}">
            <h2>{nombre}</h2>
            <p>${precio}</p>
        </div>
    '''

# Cerrar las etiquetas HTML
html_content += '''
    </div>
</body>
</html>
'''

# Guardar el HTML en un archivo
with open('productos_copilot.html', 'w') as htmlfile:
    htmlfile.write(html_content)

print("Archivo HTML generado correctamente.")

Archivo HTML generado correctamente.


#Descargar archivos generados en colab

In [20]:
import os
from google.colab import files

In [22]:
for i in os.listdir('/content/'):
 #print(i)
 files.download('/content/'+i)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>